In [1]:
from mitools import reg
from linearmodels.datasets import meps
from collections import OrderedDict
from linearmodels.iv.results import compare
import pandas as pd

# Instrumental Variables from Linearmodels

https://bashtage.github.io/linearmodels/iv/examples/advanced-examples.html

## Further Examples

### Linear Instrumental-Variables Regression

These examples follow those in **Chapter 6** of Microeconometrics Using Stata by Cameron & Trivedi.

The first step is to import the main estimator for linear IV models:

* `IV2SLS` - standard two-stage least squares

* `IVLIML` - Limited information maximum likelihood and k-class estimators

* `IVGMM` - Generalized method of moment estimation

* `IVGMMCUE` - Generalized method of moment estimation using continuously updating

## Importing Data

The data uses comes from the Medical Expenditure Panel Survey (MEPS) and includes data on out-of-pocket drug expenditure (in logs), individual characteristics, whether an individual was insured through an employer or union (a likely endogenous variable), and some candidate instruments including the percentage of income from Social Security Income, the size of the individual”s firm and whether the firm has multiple locations.

In [ ]:
data = meps.load()
data = data.dropna()
print(meps.DESCR)

Next the data – dependent, endogenous and controls – are summarized. The controls are grouped into a list to simplify model building.

In [ ]:
controls = ["totchr", "female", "age", "linc", "blhisp"]
print(data[["ldrugexp", "hi_empunion"] + controls].describe(percentiles=[]))

It is also worth examining the instruments.

In [ ]:
instruments = ["ssiratio", "lowincome", "multlc", "firmsz"]
print(data[instruments].describe(percentiles=[]))

And finally the simple correlation between the endogenous variable and the instruments. Instruments must be correlated to be relevant (but also must be exogenous, which can”t be examined using simple correlation). The correlation of `firmsz` is especially low, which might lead to the weak instruments problem if used exclusively.

In [ ]:
data[["hi_empunion"] + instruments].corr()

`add_constant` from `statsmodels` is used to simplify the process of adding a constant column to the data.

In [6]:
data["const"] = 1
controls = ["const"] + controls

### 2SLS as OLS

Before examining the IV estimators, it is worth noting that 2SLS nests the OLS estimator, so that a call to `IV2SLS` using None for the endogenous and instruments will produce OLS estimates of parameters.

The OLS estimates indicate that insurance through an employer or union leads to an **increase** in out-of-pocket drug expenditure.

In [ ]:
ivolsmod = reg.IV2SLSModel(data=data, 
                           dependent_variable="ldrugexp", 
                           independent_variables=["hi_empunion"] + controls)
res_ols = ivolsmod.fit()
print(res_ols)

### Just identified 2SLS

The just identified two-stage LS estimator uses as many instruments as endogenous variables. In this example there is one of each, using the SSI ratio as the instrument. The with the instrument, the effect of insurance through employer or union has a strong negative effect on drug expenditure.

In [ ]:
ivmod = reg.IV2SLSModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=["ssiratio"])
res_2sls = ivmod.fit()
print(res_2sls.summary)

### Multiple Instruments

Using multiple instruments only requires expanding the data array in the instruments input.

In [9]:
ivmod = reg.IV2SLSModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=["ssiratio", "multlc"])
res_2sls_robust = ivmod.fit()

### Multivariance Estimators

All estimator allow for three types of parameter covariance estimator:

* `"unadjusted"` is the classic homoskedastic estimator

* `"robust"` is robust to heteroskedasticity

* `"clustered"` allows one- or two-way clustering to account for additional sources of dependence between the model scores

* `"kernel"` produces a heteroskedasticity-autocorrelation robust covariance estimator

The default is `"robust"`.

These are all passed using the keyword input `cov_type`. Using clustered requires also passing the clustering variable(s).



In [10]:
ivmod = reg.IV2SLSModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=["ssiratio", "multlc"])
res_2sls_std = ivmod.fit(cov_type="unadjusted")

### GMM Estimation

GMM estimation can be more efficient than 2SLS when there are more than one instrument. By default, 2-step efficient GMM is used (assuming the weighting matrix is correctly specified). It is possible to iterate until convergence using the optional keyword input `iter_limit`, which is naturally 2 by default. Generally, GMM-CUE would be preferred to using multiple iterations of standard GMM.

The default weighting matrix is robust to heteroskedasticity (but not clustering).

In [11]:
ivmod = reg.IVGMMModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=["ssiratio", "multlc"])
res_gmm = ivmod.fit()

### Changing the weighting matrix structure in GMM estimation

The weighting matrix in the GMM objective function can be altered when creating the model. This example uses clustered weight by age. The covariance estimator should usually match the weighting matrix, and so clustering is also used here.

In [12]:
ivmod = reg.IVGMMModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=["ssiratio", "multlc"],
                   weight_type="clustered",
                   clusters=data.age)
res_gmm_clustered = ivmod.fit(cov_type="clustered", clusters=data.age)

### Continusouly Updating GMM

The continuously updating GMM estimator simultaneously optimizes the moment conditions and the weighting matrix. It can be more efficient (in the second order sense) than standard 2-step GMM, although it can also be fragile. Here the optional input `display` is used to produce the output of the non-linear optimizer used to estimate the parameters.

In [ ]:
ivmod = reg.IVGMMCUEModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=["ssiratio", "multlc"])
res_gmm_cue = ivmod.fit(cov_type="robust", display=True)

## Comparing Results

The function `compare` can be used to compare the results of multiple models, possibly with different variables, estimators and/or instruments. Usually a dictionary or `OrderedDict` is used to hold results since the keys are used as model names. The advantage of an `OrderedDict` is that it will preserve the order of the models in the presentation.

With the expectation of the OLS estimate, the parameter estimates are fairly consistent. Standard errors vary slightly although the conclusions reached are not sensitive to the choice of covariance estimator either. T-stats are reported in parentheses.

In [ ]:
res = OrderedDict()
res["OLS"] = res_ols
res["2SLS"] = res_2sls
res["2SLS-Homo"] = res_2sls_std
res["2SLS-Hetero"] = res_2sls_robust
res["GMM"] = res_gmm
res["GMM Cluster(Age)"] = res_gmm_clustered
res["GMM-CUE"] = res_gmm_cue
print(compare(res))

### Testing Endogeneity

The Durbin test is a classic of endogeneity which compares OLS estimates with 2SLS and exploits the fact that OLS estimates will be relatively efficient. Durbin”s test is not robust to heteroskedasticity.

In [ ]:
res_2sls.durbin()

The Wu-Hausman test is a variant of the Durbin test that uses a slightly different form.

In [ ]:
res_2sls.wu_hausman()

The test statistic can be directly replicated using the squared t-stat in a 2-stage approach where the first stage regresses the endogenous variable on the controls and instrument and the second stage regresses the dependent variable on the controls, the endogenous regressor and the residuals. If the regressor was in fact exogenous, the residuals should not be correlated with the dependent variable.

In [ ]:
step1 = reg.IV2SLSModel(data=data, 
                   dependent_variable="hi_empunion", 
                   independent_variables=['ssiratio'] + controls,
                   ).fit()
resids = step1.resids
data2 = pd.concat([data[["ldrugexp", "hi_empunion"] + controls], resids], axis=1)
step2 = reg.IV2SLSModel(data=data2, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls + ["hi_empunion", 'residual'],).fit(cov_type="unadjusted")
print(step2.tstats.residual**2)

Wooldridge”s regression-based test of exogeneity is robust to heteroskedasticity since it inherits the covariance estimator from the model. Here there is little difference.


In [ ]:
res_2sls.wooldridge_regression

Wooldridge”s score test is an alternative to the regression test, although it usually has slightly less power since it is an LM rather than a Wald type test.

In [ ]:
res_2sls.wooldridge_score

### Exogeneity Test

When there is more than one instrument (the model is overidentified), the J test can be used in GMM models to test whether the model is overidentified – in other words, whether the instruments are actually exogenous (assuming they are relevant). In the case with 2 instruments there is no evidence that against the null.

In [ ]:
res_gmm.j_stat

When all instruments are included the story changes, and some of the additional instrument (`lowincome` or `firmsz`) appear to be endogenous.

In [ ]:
ivmod = reg.IVGMMModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=instruments)
res_gmm_all = ivmod.fit()
res_gmm_all.j_stat

## Single Instrument Regressions

It can be useful to run the just identified regressions to see how the IV estimate varies by instrument. The OLS model is included for comparison. The coefficient when using lowincome is very similar to the OLS as is the $R^2$ which indicates this variable may be endogenous. The coefficient using `firmsz` is also very different, but this is probably due to the low correlation between `firmsz` and the endogenous regressor so that this is a weak instrument.

In [ ]:
od = OrderedDict()
for col in instruments:
    od[col] = reg.IV2SLSModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=[col]).fit(cov_type="robust")
od["OLS"] = res_ols
print(compare(od))

### First Stage Diagnostics

First stage diagnostics are available to assess whether the instruments appear to be credible for the endogenous regressor. The Partial F-statistic is the F-statistic for all instruments once controls have been partialed out. In the case of a single instrument, it is just the squared t-stat.

In [ ]:
print(res_2sls.first_stage)

The F-statistic actually has a $chi^2$ distribution since it is just a Wald test that all of the coefficients are 0. This breaks the “rule-of-thumb” but it can be applied by dividing the F-stat by the number of instruments.



In [ ]:
ivmod = reg.IV2SLSModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=instruments)
res_2sls_all = ivmod.fit()
print(res_2sls_all.first_stage)

## LIML

The LIML estimator and related k-class estimators can be used through `IVLIML`. LIML can have better finite sample properties if the model is not strongly identified. By default the $k$ parameter is estimated. In this dataset it is very close to 1 and to the results for LIML are similar to 2SLS (they would be exact if $k=1$).

In [ ]:
ivmod = reg.IVLIMLModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=controls, 
                   endogenous_variables=["hi_empunion"], 
                   instrument_variables=["ssiratio", "multlc"])
res_liml = ivmod.fit(cov_type="robust")
print(compare({"2SLS": res_2sls_robust, "LIML": res_liml, "GMM": res_gmm}))

The estimated value of $k$.

In [ ]:
print(res_liml.kappa)

### IV2SLS to OLS

As one final check, the “OLS” version of `IV2SLS` is compared to `statsmodels` OLS command. The parameters are identical.

In [ ]:
ivolsmod = reg.IV2SLSModel(data=data, 
                   dependent_variable="ldrugexp", 
                   independent_variables=["hi_empunion"] + controls,
                   endogenous_variables=None, 
                   instrument_variables=None)
res_ivols = ivolsmod.fit()
sm_ols = res_ols.params
sm_ols.name = "sm"
print(pd.concat([res_ivols.params, sm_ols], axis=1))

***